In [1]:
#! pip install opencv-python

     --------------------------------------- 38.2/38.2 MB 13.4 MB/s eta 0:00:00


In [1]:
import cv2
import numpy as np 

In [2]:
yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg") 
#dnn deep neural networks

In [3]:
classes=[]

In [4]:
with open("coco_1.names","r") as file:
    classes = [line.strip() for line in file.readlines()]
layer_names = yolo.getLayerNames()
# two main extraction layernames and getunconnectedoutlayer
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

In [5]:
print(classes[0])

person


In [6]:
print(classes[2])

car


In [7]:
color_Red = (0,0,255)
color_Green = (0,255,0)

In [8]:
name = "image_1.jpg"
img = cv2.imread(name)


In [9]:
height, width, channels = img.shape
# load image here 

In [10]:
blob = cv2.dnn.blobFromImage(img,0.00392, (416,416), (0,0,0), True, crop= False)
# blobfromimage remove all dependencies like light texture and it comes to the basic of basic 
#like some images are different in some places and same image is different in different place
# so to remove that dependencies we use this 

In [11]:
yolo.setInput(blob)
outputs_file = yolo.forward(output_layers)

In [12]:
class_ids  = []
confidences = [] # to remove noise like other objects 
boxes = []
for output in outputs_file:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [13]:
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
# NMS--> Non Maximum suppression When using object detection methods 
#it happens often, that the same object get’s detected multiple times
#in slightly different areas.then we use this to avoid multiple detection
# boxes which detect they detect everything like noise also
# to avaoid we use NMS
# if we not use NMS there will be multiple boxes for single object
print(class_ids)
#print(list(set(class_ids)))

print(indexes)

[16, 16, 16, 15, 16, 16, 16, 15, 15, 15, 15, 16, 15]
[1 9]


In [333]:
'''file = open("coco_1.names")
specified_lines = [16,15]
for pos, l_num in enumerate(file):

    if pos in specified_lines:
        # print the required line number
        print(l_num)'''

'file = open("coco_1.names")\nspecified_lines = [16,15]\nfor pos, l_num in enumerate(file):\n\n    if pos in specified_lines:\n        # print the required line number\n        print(l_num)'

In [14]:
# to draw boxes
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        print(boxes[i])
        print(class_ids[i])
        label = str(classes[class_ids[i]])
        #label = 'dog'
        cv2.rectangle(img,(x,y),(x + w, y + h), color_Green, 3)
        cv2.putText(img, label,(x , y + 10), cv2.FONT_HERSHEY_PLAIN,8, color_Red,8) # 8 is thickeness of font
    

[28, 26, 735, 503]
16
[698, 59, 519, 517]
15


In [15]:
print(len(boxes))
print(len(indexes))

13
2


In [17]:
cv2.imshow("image",img)
cv2.imwrite("obj_det_"+name,img) # name of new image
cv2.waitKey(0)
cv2.destroyAllWindows()